In [1]:
from definitions import data_source_path, corr_country_names, sign_level, fake_num, show_plots, \
    country_col, year_col, quarter_col, month_col, date_col, incl_countries, show_results
from helper_functions_general import read_data, select_country_year_measure, month_name_to_num, rename_order_scale, \
    downsample_month_to_quarter, quarter_to_month, upsample_quarter_to_month, get_timeframe_col, get_trans, \
    get_data_path, interpolate_series, get_table_path, get_impl_date

import matplotlib.pyplot as plt
import pandas as pd

from scipy.stats import ttest_1samp, shapiro

In [2]:
timeframe = 'm'
model = 'sc'

In [7]:
target_country = 'portugal'

# act_pred_log_diff

In [8]:
var_title = 'act_pred_log_diff'

tables_path_res = get_table_path(timeframe=timeframe, folder='results', country=target_country)
var_name = f'{model}_{target_country}_{timeframe}_{var_title}'

act_pred_log_diff = read_data(source_path=tables_path_res, file_name=var_name)
act_pred_log_diff = act_pred_log_diff.set_index(date_col)
act_pred_log_diff

act_pred_log_diff_pre = act_pred_log_diff[act_pred_log_diff.index < get_impl_date(target_country=target_country)]
act_pred_log_diff_pre

act_pred_log_diff_post = act_pred_log_diff[act_pred_log_diff.index >= get_impl_date(target_country=target_country)]
act_pred_log_diff_post

# plt.plot(act_pred_log_diff[['act', 'pred']])
# plt.plot(act_pred_log_diff['error'])

,act,pred,error
date,,,
2015-01-01,0.149161,0.006454,-0.142707
2015-02-01,0.128294,0.008277,-0.120017
2015-03-01,0.050892,-0.017029,-0.067920
2015-04-01,0.114095,0.016277,-0.097819
2015-05-01,0.001809,0.018875,0.017066
2015-06-01,0.065981,0.004921,-0.061059
2015-07-01,0.060651,0.016278,-0.044372
2015-08-01,0.043263,0.005968,-0.037295
2015-09-01,0.030482,0.025652,-0.004830


In [9]:
# pre
print('pre-treatment data')
df_pre = act_pred_log_diff_pre

print(f"mean: {df_pre['error'].mean()}")
print(f"std:  {df_pre['error'].std()}")

shap = shapiro(df_pre['error'])
if shap[1] > sign_level:
    print(f'Errors are normally distributed (p-value={round(shap[1],3)})')
else:
    print(f'Errors NOT normally distributed (p-value={round(shap[1],3)})')

ttest_res = ttest_1samp(df_pre['error'], popmean=0)
if ttest_res[1] < sign_level:
    print(f'Significant (p-value={round(ttest_res[1],3)})')
else:
    print(f'NOT significant (p-value={round(ttest_res[1],3)})')

pre-treatment data
mean: 0.015405947655621373
std:  0.09963740966309015
Errors NOT normally distributed (p-value=0.056)
Significant (p-value=0.055)


In [10]:
# post
df_post = act_pred_log_diff_post
print('post-treatment data')

print(f"mean: {df_post['error'].mean()}")
print(f"std:  {df_post['error'].std()}")
print(f"sum: {df_post['error'].sum()}")

ttest_res = ttest_1samp(df_post['error'], popmean=0)
if ttest_res[1] < sign_level:
    print(f'Significant (p-value={round(ttest_res[1],3)})')
else:
    print(f'NOT significant (p-value={round(ttest_res[1],3)})')

post-treatment data
sum: -0.1968025211061335
mean: -0.0033931469156229903
std:  0.07942071032523641
NOT significant (p-value=0.746)


# act_pred

In [28]:
var_title = 'act_pred'

tables_path_res = get_table_path(timeframe=timeframe, folder='results', country=target_country)
var_name = f'{model}_{target_country}_{timeframe}_{var_title}'

act_pred = read_data(source_path=tables_path_res, file_name=var_name)
act_pred = act_pred.set_index(date_col)
act_pred

act_pred_pre = act_pred[act_pred.index < get_impl_date(target_country=target_country)]
act_pred_pre

act_pred_post = act_pred[act_pred.index >= get_impl_date(target_country=target_country)]
act_pred_post

# plt.plot(act_pred_log_diff[['act', 'pred']])
# plt.plot(act_pred_log_diff['error'])

,act,pred,error
date,,,
2014-01-01,4.127932e+10,4.264745e+10,-1.368127e+09
2014-02-01,3.927149e+10,4.079147e+10,-1.519980e+09
2014-03-01,3.958142e+10,3.906602e+10,5.154079e+08
2014-04-01,3.252053e+10,3.378147e+10,-1.260936e+09
2014-05-01,2.964769e+10,3.233694e+10,-2.689250e+09
...,...,...,...
2019-06-01,2.726414e+10,2.848050e+10,-1.216358e+09
2019-07-01,2.723795e+10,2.805785e+10,-8.198998e+08
2019-08-01,2.623962e+10,2.760461e+10,-1.364987e+09


In [29]:
# pre
print('pre-treatment data')
df_pre = act_pred_log_diff_pre

print(f"mean: {df_pre['error'].mean()}")
print(f"std:  {df_pre['error'].std()}")

shap = shapiro(df_pre['error'])
if shap[1] > sign_level:
    print(f'Errors are normally distributed (p-value={round(shap[1],3)})')
else:
    print(f'Errors NOT normally distributed (p-value={round(shap[1],3)})')


ttest_res = ttest_1samp(df_pre['error'], popmean=0)
if ttest_res[1] < sign_level:
    print(f'Significant (p-value={round(ttest_res[1],3)})')
else:
    print(f'NOT significant (p-value={round(ttest_res[1],3)})')

pre-treatment data
mean: -0.0007944274314014444
std:  0.050511717091358696
Errors NOT normally distributed (p-value=0.083)
NOT significant (p-value=0.851)


In [30]:
# post
df_post = act_pred_post
print('post-treatment data')

print(f"sum: {df_post['error'].sum()}")
print(f"mean: {df_post['error'].mean()}")
print(f"std:  {df_post['error'].std()}")

ttest_res = ttest_1samp(df_post['error'], popmean=0)
if ttest_res[1] < sign_level:
    print(f'Significant (p-value={round(ttest_res[1],3)})')
else:
    print(f'NOT significant (p-value={round(ttest_res[1],3)})')

post-treatment data
sum: -132052335822.75134
mean: -1886461940.3250191
std:  1644242888.579198
Significant (p-value=0.0)


In [11]:
act_pred_post['error'].sum()

1326720914.030089